In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [31]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv('../train_dist.csv', index_col="pair_id")
test = pd.read_csv('../test_dist.csv', index_col="pair_id")

In [4]:
train.head()

,Unnamed: 0,name_1,name_2,is_duplicate,lemm_name_1,lemm_name_2,stopw_name_1,stopw_name_2,clean_name_1,clean_name_2,...,lemm_strcmp95,lemm_needleman_wunsch,lemm_sorensen,lemm_tversky,lemm_overlap,lemm_monge_elkan,lemm_bag,lemm_lcsseq,lemm_lcsstr,lemm_ratcliff_obershelp
pair_id,,,,,,,,,,,,,,,,,,,,,
1,0,iko industries,enormous industrial trade,0,iko industry,enormous industrial trade,iko industry,enormous industrial trade,iko,enormous,...,0.579852,-4.0,0.540541,0.370370,0.833333,0.069444,15,o industr,industr,0.486486
2,1,apcotex industries,technocraft industries,0,apcotex industry,technocraft industry,apcotex industry,technocraft industry,NaN,technocraft,...,0.726726,7.0,0.777778,0.636364,0.875000,0.054688,6,cot industry,industry,0.611111
3,2,rishichem distributors,dsa,0,rishichem distributor,dsa,rishichem distributor,dsa,rishichem distributor,NaN,...,0.498413,-16.0,0.166667,0.090909,0.666667,0.006803,19,ds,s,0.083333
4,3,powermax rubber factory,co one,0,powermax rubber factory,co one,powermax rubber factory,co one,powermax factory,NaN,...,0.474396,-14.0,0.344828,0.208333,0.833333,0.013233,18,o e,o,0.137931
5,4,tress as,longyou industries park zhejiang,0,tress a,longyou industry park zhejiang,tress,longyou industry park zhejiang,tress,longyou park,...,0.528889,-19.0,0.324324,0.193548,0.857143,0.142857,24,tr a,tr,0.216216


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 497819 entries, 1 to 497819
Data columns (total 54 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Unnamed: 0                 497819 non-null  int64  
 1   name_1                     497819 non-null  object 
 2   name_2                     497819 non-null  object 
 3   is_duplicate               497819 non-null  int64  
 4   lemm_name_1                497819 non-null  object 
 5   lemm_name_2                497819 non-null  object 
 6   stopw_name_1               497151 non-null  object 
 7   stopw_name_2               497134 non-null  object 
 8   clean_name_1               418795 non-null  object 
 9   clean_name_2               420898 non-null  object 
 10  levenstein                 497819 non-null  float64
 11  norm_levenstein            497819 non-null  float64
 12  partial_ratio              497819 non-null  float64
 13  token_sort_ratio           49

In [6]:
train.columns

Index(['Unnamed: 0', 'name_1', 'name_2', 'is_duplicate', 'lemm_name_1',
       'lemm_name_2', 'stopw_name_1', 'stopw_name_2', 'clean_name_1',
       'clean_name_2', 'levenstein', 'norm_levenstein', 'partial_ratio',
       'token_sort_ratio', 'token_set_ratio', 'WRatio', 'partial_ratio_stopw',
       'token_sort_ratio_stopw', 'token_set_ratio_stopw', 'WRatio_stopw',
       'clean_jaccard', 'clean_cosine', 'clean_mra', 'clean_jaro_winkler',
       'clean_hamming', 'clean_mlipns', 'clean_damerau_levenshtein',
       'clean_strcmp95', 'clean_needleman_wunsch', 'clean_sorensen',
       'clean_tversky', 'clean_overlap', 'clean_monge_elkan', 'clean_bag',
       'clean_lcsseq', 'clean_lcsstr', 'clean_ratcliff_obershelp',
       'lemm_jaccard', 'lemm_cosine', 'lemm_mra', 'lemm_jaro_winkler',
       'lemm_hamming', 'lemm_mlipns', 'lemm_damerau_levenshtein',
       'lemm_strcmp95', 'lemm_needleman_wunsch', 'lemm_sorensen',
       'lemm_tversky', 'lemm_overlap', 'lemm_monge_elkan', 'lemm_bag',
   

# Параметры отбора

In [7]:
features_name = ['levenstein', 'norm_levenstein', 'partial_ratio',
       'token_sort_ratio', 'token_set_ratio', 'WRatio', 'partial_ratio_stopw',
       'token_sort_ratio_stopw', 'token_set_ratio_stopw', 'WRatio_stopw',
       'clean_jaccard', 'clean_cosine', 'clean_mra', 'clean_jaro_winkler',
       'clean_hamming', 'clean_mlipns', 'clean_damerau_levenshtein',
       'clean_strcmp95', 'clean_needleman_wunsch', 'clean_sorensen',
       'clean_tversky', 'clean_overlap', 'clean_monge_elkan', 'clean_bag', 'clean_ratcliff_obershelp',
       'lemm_jaccard', 'lemm_cosine', 'lemm_mra', 'lemm_jaro_winkler',
       'lemm_hamming', 'lemm_mlipns', 'lemm_damerau_levenshtein',
       'lemm_strcmp95', 'lemm_needleman_wunsch', 'lemm_sorensen',
       'lemm_tversky', 'lemm_overlap', 'lemm_monge_elkan', 'lemm_bag', 'lemm_ratcliff_obershelp']

print(len(features_name))

40


In [ ]:
num_features = 20

# Подготовка данных

In [8]:
from sklearn.preprocessing import MinMaxScaler

X = train[features_name]
X_norm = MinMaxScaler().fit_transform(X)

y = train['is_duplicate']

In [9]:
X_norm

array([[0.11811024, 0.6       , 0.71      , ..., 0.03472222, 0.12      ,
        0.48648649],
       [0.07086614, 0.40909091, 0.72      , ..., 0.02734375, 0.048     ,
        0.61111111],
       [0.15748031, 0.90909091, 0.67      , ..., 0.00340136, 0.152     ,
        0.08333333],
       ...,
       [0.11023622, 0.42424242, 0.7       , ..., 0.01331497, 0.056     ,
        0.64615385],
       [0.1023622 , 0.54166667, 0.59      , ..., 0.01736111, 0.056     ,
        0.47826087],
       [0.11811024, 0.55555556, 0.67      , ..., 0.02539062, 0.128     ,
        0.41860465]])

# Recursive Feature Elimination

In [10]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [13]:
rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=num_features, step=1, verbose=5)
rfe_selector.fit(X_norm, y)

Fitting estimator with 40 features.


/home/agladyshev/anaconda3/envs/unn-nlp/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Fitting estimator with 39 features.


/home/agladyshev/anaconda3/envs/unn-nlp/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Fitting estimator with 38 features.


...

Fitting estimator with 22 features.


/home/agladyshev/anaconda3/envs/unn-nlp/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Fitting estimator with 21 features.


/home/agladyshev/anaconda3/envs/unn-nlp/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/agladyshev/anaconda3/envs/unn-nlp/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn

RFE(estimator=LogisticRegression(), n_features_to_select=20, verbose=5)

In [14]:
rfe_support = rfe_selector.get_support()
rfe_feature = X.loc[:,rfe_support].columns.tolist()
print(str(len(rfe_feature)), 'selected features')

20 selected features


In [15]:
rfe_feature

['levenstein',
 'norm_levenstein',
 'partial_ratio',
 'token_set_ratio',
 'WRatio',
 'token_sort_ratio_stopw',
 'clean_jaccard',
 'clean_jaro_winkler',
 'clean_hamming',
 'clean_damerau_levenshtein',
 'clean_strcmp95',
 'clean_sorensen',
 'clean_tversky',
 'clean_bag',
 'lemm_cosine',
 'lemm_jaro_winkler',
 'lemm_hamming',
 'lemm_damerau_levenshtein',
 'lemm_strcmp95',
 'lemm_bag']

# Lasso: SelectFromModel

In [38]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

embeded_lr_selector = SelectFromModel(LogisticRegression(penalty="l1", solver='liblinear'), max_features=num_features)
embeded_lr_selector.fit(X_norm, y)

SelectFromModel(estimator=LogisticRegression(penalty='l1', solver='liblinear'),
                max_features=20)

In [39]:
embeded_lr_support = embeded_lr_selector.get_support()
embeded_lr_feature = X.loc[:,embeded_lr_support].columns.tolist()
print(str(len(embeded_lr_feature)), 'selected features')

20 selected features


In [40]:
embeded_lr_feature

['levenstein',
 'norm_levenstein',
 'partial_ratio',
 'token_sort_ratio',
 'token_set_ratio',
 'WRatio',
 'token_sort_ratio_stopw',
 'clean_jaccard',
 'clean_cosine',
 'clean_jaro_winkler',
 'clean_hamming',
 'clean_strcmp95',
 'clean_needleman_wunsch',
 'clean_tversky',
 'lemm_cosine',
 'lemm_jaro_winkler',
 'lemm_hamming',
 'lemm_damerau_levenshtein',
 'lemm_needleman_wunsch',
 'lemm_bag']

# Tree-based: SelectFromModel

In [21]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100), max_features=num_features)
embeded_rf_selector.fit(X, y)

SelectFromModel(estimator=RandomForestClassifier(), max_features=20)

In [22]:
embeded_rf_support = embeded_rf_selector.get_support()
embeded_rf_feature = X.loc[:,embeded_rf_support].columns.tolist()
print(str(len(embeded_rf_feature)), 'selected features')

13 selected features


In [23]:
embeded_rf_feature

['partial_ratio',
 'token_set_ratio',
 'WRatio',
 'partial_ratio_stopw',
 'token_set_ratio_stopw',
 'clean_jaro_winkler',
 'clean_strcmp95',
 'clean_needleman_wunsch',
 'lemm_jaro_winkler',
 'lemm_hamming',
 'lemm_strcmp95',
 'lemm_needleman_wunsch',
 'lemm_overlap']

# LightGBM

In [25]:
!pip install lightgbm

     |████████████████████████████████| 1.8 MB 1.1 MB/s eta 0:00:01


In [27]:
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier

lgbc=LGBMClassifier(n_estimators=500, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,
            reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)

embeded_lgb_selector = SelectFromModel(lgbc, max_features=num_features)
embeded_lgb_selector.fit(X, y)

SelectFromModel(estimator=LGBMClassifier(colsample_bytree=0.2,
                                         learning_rate=0.05,
                                         min_child_weight=40,
                                         min_split_gain=0.01, n_estimators=500,
                                         num_leaves=32, reg_alpha=3,
                                         reg_lambda=1),
                max_features=20)

In [28]:
embeded_lgb_support = embeded_lgb_selector.get_support()
embeded_lgb_feature = X.loc[:,embeded_lgb_support].columns.tolist()
print(str(len(embeded_lgb_feature)), 'selected features')

17 selected features


In [29]:
embeded_lgb_feature

['norm_levenstein',
 'token_sort_ratio',
 'token_set_ratio',
 'WRatio',
 'token_sort_ratio_stopw',
 'token_set_ratio_stopw',
 'WRatio_stopw',
 'clean_cosine',
 'clean_jaro_winkler',
 'clean_sorensen',
 'clean_overlap',
 'lemm_jaccard',
 'lemm_hamming',
 'lemm_needleman_wunsch',
 'lemm_overlap',
 'lemm_monge_elkan',
 'lemm_ratcliff_obershelp']

# Сравнение

In [41]:
pd.set_option('display.max_rows', None)

# put all selection together
feature_selection_df = pd.DataFrame({'Feature':features_name,
                                     'RFE': rfe_support,
                                     'Logistics': embeded_lr_support,
                                     'Random Forest': embeded_rf_support,
                                     'LightGBM': embeded_lgb_support})

# count the selected times for each feature
feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)

feature_selection_df = feature_selection_df.sort_values(['Total', 'Feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df) + 1)
feature_selection_df.head(num_features)

,Feature,RFE,Logistics,Random Forest,LightGBM,Total
1,token_set_ratio,True,True,True,True,4
2,lemm_hamming,True,True,True,True,4
3,clean_jaro_winkler,True,True,True,True,4
4,WRatio,True,True,True,True,4
5,token_sort_ratio_stopw,True,True,False,True,3
6,partial_ratio,True,True,True,False,3
7,norm_levenstein,True,True,False,True,3
8,lemm_needleman_wunsch,False,True,True,True,3
9,lemm_jaro_winkler,True,True,True,False,3
10,clean_strcmp95,True,True,True,False,3


In [42]:
feature_selection_df['Feature'].tolist()

['token_set_ratio',
 'lemm_hamming',
 'clean_jaro_winkler',
 'WRatio',
 'token_sort_ratio_stopw',
 'partial_ratio',
 'norm_levenstein',
 'lemm_needleman_wunsch',
 'lemm_jaro_winkler',
 'clean_strcmp95',
 'token_sort_ratio',
 'token_set_ratio_stopw',
 'levenstein',
 'lemm_strcmp95',
 'lemm_overlap',
 'lemm_damerau_levenshtein',
 'lemm_cosine',
 'lemm_bag',
 'clean_tversky',
 'clean_sorensen',
 'clean_needleman_wunsch',
 'clean_jaccard',
 'clean_hamming',
 'clean_cosine',
 'partial_ratio_stopw',
 'lemm_ratcliff_obershelp',
 'lemm_monge_elkan',
 'lemm_jaccard',
 'clean_overlap',
 'clean_damerau_levenshtein',
 'clean_bag',
 'WRatio_stopw',
 'lemm_tversky',
 'lemm_sorensen',
 'lemm_mra',
 'lemm_mlipns',
 'clean_ratcliff_obershelp',
 'clean_mra',
 'clean_monge_elkan',
 'clean_mlipns']